# AirPump failures



In [1]:
#Fastparquet
import pandas as pd
import numpy as np

import os
import zipfile

import torch


In [2]:
!pip install fastparquet

In [3]:
with zipfile.ZipFile('data.zip') as z:
    for filename in z.namelist():
        if not os.path.isdir(filename):
            if '.parquet' in filename:
                data = pd.read_parquet(z.open(filename), engine='fastparquet') 
            if '.csv' in filename:
                labels = pd.read_csv(z.open(filename))  


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18466364 entries, 0 to 18466363
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   MachineId      object 
 1   MeasurementId  int64  
 2   Pressure       float64
dtypes: float64(1), int64(1), object(1)
memory usage: 422.7+ MB


In [5]:
data.head()

,MachineId,MeasurementId,Pressure
0,0_0_0,0,0.0
1,0_0_0,0,0.0
2,0_0_0,0,0.0
3,0_0_0,0,0.0
4,0_0_0,0,0.0


In [6]:
data['MeasurementId'].unique().size

8836

In [7]:
len(data['MachineId'].unique())

556

Celkem pracujeme s 8836 měřenými cykly a 556 zařízeními, která monitorujeme.

In [8]:
#data.to_csv('out.csv',index=False)

In [9]:
labels.head()

,MachineId,MeasurementId,PumpFailed,SlowStart,SlowEnd
0,0_0_0,0,False,False,False
1,0_0_1,-1,NaN,NaN,NaN
2,0_0_2,0,False,False,False
3,0_0_3,0,True,False,False
4,0_1_0,-1,NaN,NaN,NaN


In [10]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27385 entries, 0 to 27384
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MachineId      27385 non-null  object
 1   MeasurementId  27385 non-null  int64 
 2   PumpFailed     26900 non-null  object
 3   SlowStart      19300 non-null  object
 4   SlowEnd        19300 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


Z předchozích řádků si můžeme všimnout, že v příznaku 'PumpFailed' máme i hodnoty null. Je potřeba si rozmyslet jak s těmito hodnotami budeme pracovat.


Nabízí se dva přístupy. Zjistíme o jaké cykly se jedná a v závislosti na této informaci můžeme tyto hodnoty vyfiltrovat. Pak bychom při predikci selhání zkoumali zda uvedený cyklus je podobný a rovnou určili jeho hodnotu na null. Tohle bychom mohli provést pokud bychom si byli jistí, že daný cyklus bude vždy takto označen v reálném provozu.


Další přístup je nahrazení hodnoty null za jinou. Jelikož se jedná o klasifikaci a budeme hodnoty True, False přepisovat do číselného označení {0, 1}, pak i hodnota null bude popsána unikátním číslem a nemusíme tento případ dále zkoumat.


Podíváme se nyní o které cykly se jedná. I když budeme postupovat podle druhé možnosti, tak nám toto může posloužit při evaulaci.

In [11]:
df_nan = labels[labels['PumpFailed'].isnull()]

In [12]:
df_nan['MeasurementId'].unique()

array([-1])

Ověříme si tvrzení zda u všech měření, která mají 'MeasurementId' == -1, není definováno zda došlo/nedošlo k selhání přístroje.

In [13]:
df_nan['MachineId'].unique().size == data[data['MeasurementId'] == -1]['MachineId'].unique().shape[0]

True

Ověřili jsme si tvrzení a mohli bychom tedy použít možnost jedna, o které jsme psali výše. Pro jistotu jevšak vhodnější i tak zvolit druhou možnost, kdy hodnoty null nahradíme jinou hodnotou, protože nikde v zadání nebylo uvedeno, že by tohle tvrzení by mělo vždy platit. Tvrzení platí pouze pro poskytnutá data.  

Vrhneme se tedy na úpravu datasetu labels, kde přepíšeme hodnoty na unikátní číselné. 

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
encoded_labels = le.fit_transform(labels.PumpFailed)

In [15]:
le.classes_

array([False, True, nan], dtype=object)

In [16]:
labels["Labels"] = encoded_labels

In [17]:
labels

,MachineId,MeasurementId,PumpFailed,SlowStart,SlowEnd,Labels
0,0_0_0,0,False,False,False,0
1,0_0_1,-1,NaN,NaN,NaN,2
2,0_0_2,0,False,False,False,0
3,0_0_3,0,True,False,False,1
4,0_1_0,-1,NaN,NaN,NaN,2
...,...,...,...,...,...,...
27380,10_0_2,8833,False,False,False,0
27381,10_0_3,8833,False,False,False,0
27382,10_0_0,8834,False,False,False,0
27383,10_0_1,8834,False,False,True,0


In [18]:
labels[labels.MeasurementId == -1]

,MachineId,MeasurementId,PumpFailed,SlowStart,SlowEnd,Labels
1,0_0_1,-1,NaN,NaN,NaN,2
4,0_1_0,-1,NaN,NaN,NaN,2
6,0_1_2,-1,NaN,NaN,NaN,2
7,0_1_3,-1,NaN,NaN,NaN,2
9,1_0_1,-1,NaN,NaN,NaN,2
...,...,...,...,...,...,...
25788,15_0_2,-1,NaN,NaN,NaN,2
25789,15_0_3,-1,NaN,NaN,NaN,2
25792,15_0_1,-1,NaN,NaN,NaN,2
27018,11_2_2,-1,NaN,NaN,NaN,2


Mapu použijeme po predikci k převedení zpět na původní hodnoty.

In [19]:
data.index

RangeIndex(start=0, stop=18466364, step=1)

In [20]:
d_freq_uniq = data.groupby(['MachineId', 'MeasurementId']).size().reset_index(name='Freq')

In [21]:
d_freq_uniq

,MachineId,MeasurementId,Freq
0,0_0_0,-1,101711
1,0_0_0,0,368
2,0_0_0,215,732
3,0_0_0,237,589
4,0_0_0,353,349
...,...,...,...
27380,9_5_3,4253,337
27381,9_5_3,4254,478
27382,9_5_3,4255,436
27383,9_5_3,4256,549


In [22]:
d_freq_uniq.Freq.max()

185476

In [23]:
d_freq_uniq.Freq.median()

438.0

In [24]:
d_freq_uniq[d_freq_uniq.Freq == d_freq_uniq.Freq.max()]

,MachineId,MeasurementId,Freq
4930,0_2_0,-1,185476


In [25]:
d_freq_uniq[d_freq_uniq.MeasurementId == -1]

,MachineId,MeasurementId,Freq
0,0_0_0,-1,101711
570,0_0_1,-1,90267
1155,0_0_2,-1,82389
1753,0_0_3,-1,68346
2378,0_1_0,-1,116338
...,...,...,...
26657,9_4_3,-1,10012
26822,9_5_0,-1,12648
26959,9_5_1,-1,11792
27097,9_5_2,-1,9674


Nyní jsme zjistili, že měření, kde 'MeasurementId' je rovný -1, mají nejrozsáhlejší dobu měření. Jejich hodnoty v příznacích v datasetu labels jsou ve většině null. Z tohoto můžeme vyvodit, že jejich přítomnost v datasetu nám může komplikovat volbu vhodného modelu a tím pádem je vhodné je vyřadit z obou datasetů pro učení modelů.

Konvoluční sítě například vyžadují, aby tensory měli stejné rozměry. Museli bychom, tak ostatní měření doplnit o nuly na konci. Již teď máme rozsáhlý dataset a další rozšíření je nežádoucí.

In [26]:
df_nan = df_nan[['MachineId', 'MeasurementId']]

In [27]:
cols = ['MachineId', 'MeasurementId']

df_ = df_nan.set_index(cols)
data_ = data.set_index(cols)

data = data[~data_.index.isin(df_.index)]
data

,MachineId,MeasurementId,Pressure
0,0_0_0,0,0.0
1,0_0_0,0,0.0
2,0_0_0,0,0.0
3,0_0_0,0,0.0
4,0_0_0,0,0.0
...,...,...,...
18466359,10_0_3,8834,0.0
18466360,10_0_3,8834,0.0
18466361,10_0_3,8834,0.0
18466362,10_0_3,8834,0.0


In [28]:
data[data.MeasurementId == -1]

,MachineId,MeasurementId,Pressure


In [29]:
df_ = df_nan.set_index(cols)
data_ = labels.set_index(cols)

labels = labels[~data_.index.isin(df_.index)]
labels

,MachineId,MeasurementId,PumpFailed,SlowStart,SlowEnd,Labels
0,0_0_0,0,False,False,False,0
2,0_0_2,0,False,False,False,0
3,0_0_3,0,True,False,False,1
5,0_1_1,1,True,False,False,1
8,1_0_0,2,False,True,False,0
...,...,...,...,...,...,...
27380,10_0_2,8833,False,False,False,0
27381,10_0_3,8833,False,False,False,0
27382,10_0_0,8834,False,False,False,0
27383,10_0_1,8834,False,False,True,0


In [30]:
labels[labels.MeasurementId == -1]

,MachineId,MeasurementId,PumpFailed,SlowStart,SlowEnd,Labels


Podařilo se nám smazat všechny záznamy, kde příznak 'MeasurementId' byl roven -1.

In [31]:
df_Measurement = data[['MachineId', 'MeasurementId']].drop_duplicates().reset_index().drop(columns='index')
df_Measurement

,MachineId,MeasurementId
0,0_0_0,0
1,0_0_2,0
2,0_0_3,0
3,0_1_1,1
4,1_0_0,2
...,...,...
26895,10_0_2,8833
26896,10_0_3,8833
26897,10_0_0,8834
26898,10_0_1,8834


Ujistili jsme se, že počet měření je stejný v obou datasetech. Nyní ověříme zda je posloupnost měření stejně seřazena podle zařízení a id měření.

In [32]:
df_Measurement.equals(labels[['MachineId', 'MeasurementId']].drop_duplicates().reset_index().drop(columns='index'))

True



---



---



Dále budume používat pouze část měření, kdy pumpa začíná vyvíjet tlak.

In [33]:
v = data.groupby(['MachineId', 'MeasurementId']).Pressure.apply(lambda x: x.tolist())

In [34]:
v

MachineId  MeasurementId
0_0_0      0                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
           215              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
           237              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
           353              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
           411              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                                                  ...                        
9_5_3      4253             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
           4254             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
           4255             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
           4256             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
           4257             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: Pressure, Length: 26900, dtype: object

In [35]:
v_list = v.tolist()

In [36]:
len(v_list[0])

368

In [37]:
df = pd.DataFrame(v_list, index=v.index)\
       .rename(columns=lambda x: x + 1)\
       .add_prefix('Pressure-')\
       .reset_index()

       #https://stackoverflow.com/questions/47991005/pandas-combine-group-by-and-rows-to-columns

In [38]:
df

,MachineId,MeasurementId,Pressure-1,Pressure-2,Pressure-3,Pressure-4,Pressure-5,Pressure-6,Pressure-7,Pressure-8,...,Pressure-1190,Pressure-1191,Pressure-1192,Pressure-1193,Pressure-1194,Pressure-1195,Pressure-1196,Pressure-1197,Pressure-1198,Pressure-1199
0,0_0_0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0_0_0,215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0_0_0,237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0_0_0,353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0_0_0,411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26895,9_5_3,4253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26896,9_5_3,4254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26897,9_5_3,4255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26898,9_5_3,4256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Nyní budeme mít jednoduší práci s úpravou dat. Sloupce, které budou obsahovat pouze nuly můžeme smazat a tak nám vznikne zarovnaný datatset pro tesory.

In [39]:
df = df.fillna(0.0)

In [40]:
df = df.drop(columns=df.columns[(df == 0.0).all()])

In [41]:
df

,MachineId,MeasurementId,Pressure-35,Pressure-36,Pressure-37,Pressure-38,Pressure-39,Pressure-40,Pressure-41,Pressure-42,...,Pressure-1112,Pressure-1113,Pressure-1114,Pressure-1115,Pressure-1116,Pressure-1117,Pressure-1118,Pressure-1119,Pressure-1120,Pressure-1121
0,0_0_0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0_0_0,215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0_0_0,237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0_0_0,353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0_0_0,411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26895,9_5_3,4253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26896,9_5_3,4254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26897,9_5_3,4255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26898,9_5_3,4256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df[df['Pressure-1121'] > 0]

,MachineId,MeasurementId,Pressure-35,Pressure-36,Pressure-37,Pressure-38,Pressure-39,Pressure-40,Pressure-41,Pressure-42,...,Pressure-1112,Pressure-1113,Pressure-1114,Pressure-1115,Pressure-1116,Pressure-1117,Pressure-1118,Pressure-1119,Pressure-1120,Pressure-1121
21269,4_2_3,6889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.264111,0.248136,0.221512,0.161874,0.160809,0.123536,0.089457,0.071353,0.054313,0.048988


Vidíme, že memá pořád problém s tím, že v některých řádcích máme přebytečné nuly. Zatím se tímto zabývat nebudeme, protože nám momentálně stačí, že máme data zarovnaná na stejnou velikost.

Přesuneme se k vytvoření sérií a následnému rozdělení do množin pro trénování.

In [43]:
df["Machine_Measurement"] = df.MachineId.copy().astype(str) + '_' + df.MeasurementId.copy().astype(str)
df

,MachineId,MeasurementId,Pressure-35,Pressure-36,Pressure-37,Pressure-38,Pressure-39,Pressure-40,Pressure-41,Pressure-42,...,Pressure-1113,Pressure-1114,Pressure-1115,Pressure-1116,Pressure-1117,Pressure-1118,Pressure-1119,Pressure-1120,Pressure-1121,Machine_Measurement
0,0_0_0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_0_0_0
1,0_0_0,215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_0_0_215
2,0_0_0,237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_0_0_237
3,0_0_0,353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_0_0_353
4,0_0_0,411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_0_0_411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26895,9_5_3,4253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9_5_3_4253
26896,9_5_3,4254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9_5_3_4254
26897,9_5_3,4255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9_5_3_4255
26898,9_5_3,4256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9_5_3_4256


In [44]:
labels["Machine_Measurement"] = labels.MachineId.copy().astype(str) + '_' + labels.MeasurementId.copy().astype(str)
labels

/var/folders/77/56jxhgt56d32np91f9n3grk80000gn/T/ipykernel_12799/2640393424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["Machine_Measurement"] = labels.MachineId.copy().astype(str) + '_' + labels.MeasurementId.copy().astype(str)


,MachineId,MeasurementId,PumpFailed,SlowStart,SlowEnd,Labels,Machine_Measurement
0,0_0_0,0,False,False,False,0,0_0_0_0
2,0_0_2,0,False,False,False,0,0_0_2_0
3,0_0_3,0,True,False,False,1,0_0_3_0
5,0_1_1,1,True,False,False,1,0_1_1_1
8,1_0_0,2,False,True,False,0,1_0_0_2
...,...,...,...,...,...,...,...
27380,10_0_2,8833,False,False,False,0,10_0_2_8833
27381,10_0_3,8833,False,False,False,0,10_0_3_8833
27382,10_0_0,8834,False,False,False,0,10_0_0_8834
27383,10_0_1,8834,False,False,True,0,10_0_1_8834


### Klasifikační model:
Nyní natrénujeme model na našich datech pomocí klasifikační metody rozhodujícího stromu. Následně zobrazíme přesnost s jakou náš model určí pravděpodobnost selhání pumpy na validačních a trénovacích datech.

In [45]:
rd_seed = 333

from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(df, labels.Labels, test_size=0.4, random_state=rd_seed) 
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size=0.4, random_state=rd_seed) 

In [46]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'max_depth': range(3,10), 
    'criterion': ['entropy', 'gini']
}
param_comb = ParameterGrid(param_grid)

val_acc = []
train_acc = []

from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
for params in param_comb:
    clf = DecisionTreeClassifier(max_depth=params['max_depth'], criterion = params['criterion'])
    clf.fit(df,labels.Labels)
    val_acc.append(metrics.accuracy_score(Yval,clf.predict(Xval)))
    train_acc.append(metrics.accuracy_score(Ytrain, clf.predict(Xtrain)))
    

best_params = param_comb[np.argmax(val_acc)]

dt = DecisionTreeClassifier(max_depth=best_params['max_depth'], criterion=best_params['criterion'])
dt.fit(Xtrain, Ytrain)
val = metrics.accuracy_score(Yval, dt.predict(Xval))*100
test = metrics.accuracy_score(Ytest, dt.predict(Xtest))*100
print("max_depth: ",best_params['max_depth'], "criterion: ", best_params['criterion'])
print('Přesnost na validačních datech: %.2f' % val,'%')
print('Přesnost na testovacích datech: %.2f' % test,'%')

max_depth:  9 criterion:  gini
Přesnost na validačních datech: 88.89 %
Přesnost na testovacích datech: 89.62 %


Přesnost modelu na nových datech by měla být zhruba 89%. 

Dále budeme pokračovat v úpravě dat, tak abychom mohli použít 1D konvoluční síť.

In [47]:
COLUMNS_PRESSUERE = df.columns[2:-1]
COLUMNS_PRESSUERE

Index(['Pressure-35', 'Pressure-36', 'Pressure-37', 'Pressure-38',
       'Pressure-39', 'Pressure-40', 'Pressure-41', 'Pressure-42',
       'Pressure-43', 'Pressure-44',
       ...
       'Pressure-1112', 'Pressure-1113', 'Pressure-1114', 'Pressure-1115',
       'Pressure-1116', 'Pressure-1117', 'Pressure-1118', 'Pressure-1119',
       'Pressure-1120', 'Pressure-1121'],
      dtype='object', length=1087)

In [48]:
sequences = []

for Machine_Measurement, group in df.groupby("Machine_Measurement"):
    sequences_features = group[COLUMNS_PRESSUERE]
    
    sequences.append(sequences_features)


In [49]:
sequences[0]

,Pressure-35,Pressure-36,Pressure-37,Pressure-38,Pressure-39,Pressure-40,Pressure-41,Pressure-42,Pressure-43,Pressure-44,...,Pressure-1112,Pressure-1113,Pressure-1114,Pressure-1115,Pressure-1116,Pressure-1117,Pressure-1118,Pressure-1119,Pressure-1120,Pressure-1121
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
for n in range(0, len(sequences)):
    sequences[n] = sequences[n].melt(
        var_name="time_stemp",
        value_name="Pressure")
    sequences[n] = sequences[n].drop(columns=["time_stemp"])


In [51]:
sequences[0]

,Pressure
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1082,0.0
1083,0.0
1084,0.0
1085,0.0


In [52]:
len(sequences)

26900

In [53]:
Labels = pd.DataFrame()
Labels["Labels"] = labels.Labels
Labels

,Labels
0,0
2,0
3,1
5,1
8,0
...,...
27380,0
27381,0
27382,0
27383,0


In [54]:
seq = np.array(sequences)

In [55]:
seq[0]

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])

In [56]:
lab = np.array(Labels)
lab

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [57]:

list_of_arrays = [np.array(df) for df in sequences]

In [58]:
list_of_arrays[0]

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

#### Vytvoření DataLoaderu pro trénování a testování:

In [59]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader


x_train, x_test, y_train, y_test = train_test_split(list_of_arrays, lab, test_size=0.3, random_state=0)

train_ds = TensorDataset(torch.tensor(np.stack(x_train), dtype=torch.float32), torch.tensor(np.stack(y_train), dtype=torch.int64)) #torch.Tensor(y_train).long())
test_ds = TensorDataset(torch.tensor(np.stack(x_test), dtype=torch.float32), torch.tensor(np.stack(y_test), dtype=torch.int64).long())

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=128, shuffle=False)

Nedokončené:

#### CNN model:

In [60]:
import torch.nn as nn

In [61]:
import torch.nn.functional as F

In [62]:
class TimeSeriesCnnModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels = 1087, out_channels = 512, kernel_size = 1)
        self.pool1 = nn.MaxPool1d(kernel_size = 1, stride = 2)
        
        self.conv2 = nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 1)
        
        
        self.conv3 = nn.Conv1d(256, 256, 1)
        self.pool3 = nn.MaxPool1d(kernel_size = 1, stride = 2)
        
        self.conv4 = nn.Conv1d(256, 128, 1)
        self.pool4 = nn.MaxPool1d(kernel_size = 1, stride = 2)
        
        self.conv5 = nn.Conv1d(128, 64, 1)
        self.pool5 = nn.MaxPool1d(kernel_size = 1, stride = 2)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        
        x = F.relu(self.conv4(x))
        x = self.pool4(x)
        
        x = F.relu(self.conv5(x))
        x = self.pool5(x)
        
        x = x.flatten(start_dim = 1)
        
        return x



In [63]:
model = TimeSeriesCnnModel()
model

TimeSeriesCnnModel(
  (conv1): Conv1d(1087, 512, kernel_size=(1,), stride=(1,))
  (pool1): MaxPool1d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
  (pool3): MaxPool1d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
  (pool4): MaxPool1d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
  (pool5): MaxPool1d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [64]:
for sequences, labels in train_dl:
    out = model(sequences)
    _, preds = torch.max(out, dim=1)
    print(preds, labels)

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
     

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

...

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

In [65]:
for sequences, labels in test_dl:
    out = model(sequences)
    _, preds = torch.max(out, dim=1)
    print(preds, labels)

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

...

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        



---



---


